In [ ]:
from azureml.core import Workspace, Experiment
from azureml.core.compute_target import ComputeTargetException
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.workspace import Workspace, Dataset


ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="banks_analysis_tuning")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

In [ ]:
cluster_name = 'mycluster'
cluster_min_nodes = 0
cluster_max_nodes = 4
cluster_sku = 'STANDARD_D2_V2'

# create the compute cluster if it does not already exist
try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print(f'Found existing compute target {cluster_name}.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size=cluster_sku, min_nodes=cluster_min_nodes, max_nodes=cluster_max_nodes)
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)
    compute_target.wait_for_completion(show_output=True)
    print(f'Compute target {cluster_name} has been created.')

In [ ]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
param_sampling = RandomParameterSampling({
    '--C': uniform(0.01, 1.0),
    '--max_iter': uniform(1, 100)
})

# Specify a Policy
early_termination_policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1, delay_evaluation=5)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(source_directory='./training',
                      script='train.py',
                      arguments=['--C', '--max_iter'],
                      environment=sklearn_env)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src,
                                     hyperparameter_sampling=param_sampling,
                                     policy=early_termination_policy,
                                     primary_metric_name='accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=12,
                                     max_concurrent_runs=4,
                                     compute_target='mycluster')

In [ ]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdrive_run = exp.submit(config=hyperdrive_config)

RunDetails(hyperdrive_run).show()

In [ ]:
import joblib
# Get your best run and save the model from that run.

# Get the best run from the experiment
best_run = hyperdrive_run.get_best_run_by_primary_metric()

# Download the best run's model file to the local file system
model_file_name = 'best_model.pkl'
best_run.download_file(name='outputs/' + model_file_name, output_file_path=model_file_name)

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

url = 'https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'

# create a TabularDatasetFactory object and use it to create a TabularDataset
datastore = ws.get_default_datastore()
dataset = Dataset.Tabular.from_delimited_files(path=url)
df = dataset.to_pandas_dataframe()


In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(df)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task="classification",
    primary_metric="accuracy",
    X=x,
    y=y,
    n_cross_validations=5)

In [2]:
# Submit your automl run

exp_2 = Experiment(ws, "automl_test_experiment")
run = exp_2.submit(config=automl_config, show_output=True)

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###